## mnist_cnn 

### GPU 사용을 위한 환경 세팅
- 런타임 => 런타임유형변경 => 하드웨어가속기 => GPU로 설정

### GUI 확인
![GUI](https://user-images.githubusercontent.com/11758940/84726212-581efd00-afc7-11ea-8166-dd4dda7b8e28.png)

In [1]:
# 현재 나에게 할당된 GPU 확인하기
! nvidia-smi

Sun Dec  5 08:18:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.76       Driver Version: 496.76       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:0A:00.0  On |                  N/A |
| 31%   29C    P8    29W / 350W |   1096MiB / 12288MiB |     16%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



c:\python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

## >> 버전 1.0
![대체 텍스트](https://user-images.githubusercontent.com/11758940/84732153-e306f400-afd5-11ea-8524-55e4f13e749b.png)

In [7]:
## 버전 1.0
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN,self).__init__() # 이부분 빼먹기 쉬운데 조심할 것

    self.conv1 = torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
    self.relu1 = torch.nn.ReLU()
    self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv2 = torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
    self.relu2 = torch.nn.ReLU()
    self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2) 

    self.fc = torch.nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.kaiming_uniform(self.fc.weight)

  def forward(self,x):
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.maxpool1(out)
    out = self.conv2(out)
    out = self.relu2(out)
    out = self.maxpool2(out)
    out = out.view(out.size(0),-1)     # out = Batch(0) x 7(1) x 7(2) x 64(3)
    out = self.fc(out)
    return out

## >> 버전 2.0
![대체 텍스트](https://user-images.githubusercontent.com/11758940/84732337-63c5f000-afd6-11ea-9e7b-3c4c88785213.png)

In [8]:
# 버전 2.0
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN,self).__init__() # 이부분 빼먹기 쉬운데 조심할 것

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    ) 

    self.fc = torch.nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.kaiming_uniform(self.fc.weight)

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1)     # out = Batch x 7 x 7 x 64
    out = self.fc(out)
    return out

In [9]:
model = CNN().to(device)

C:\Users\khc\AppData\Local\Temp/ipykernel_23792/3635404650.py:19: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  torch.nn.init.kaiming_uniform(self.fc.weight)


In [10]:
# mnist_dnn 예제와 동일
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# mnist_dnn 예제와 동일: X데이터 View로 변환하지 않는 부분만 다름
# 주의: 2D => 1D로 변경할 필요 없음


#train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.


c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch:    1] cost = 0.225696966
[Epoch:    2] cost = 0.0629915968
[Epoch:    3] cost = 0.0461295135
[Epoch:    4] cost = 0.0372560881
[Epoch:    5] cost = 0.0308627859
[Epoch:    6] cost = 0.0256840959
[Epoch:    7] cost = 0.0213675015
[Epoch:    8] cost = 0.0179518107
[Epoch:    9] cost = 0.015742125
[Epoch:   10] cost = 0.0127761625
[Epoch:   11] cost = 0.010358735
[Epoch:   12] cost = 0.0096156057
[Epoch:   13] cost = 0.00872339122
[Epoch:   14] cost = 0.0061495211
[Epoch:   15] cost = 0.00542137213
Learning Finished!


In [12]:
# # mnist_dnn 예제와 동일: X데이터 View로 변환하지 않는 부분만 다름
# 주의: 2D => 1D로 변경할 필요 없음

# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

c:\python39\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\python39\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9863999485969543
